In [11]:
#장르별 리스트,독점리스트 따로구성
def kakaopage_title_extraction():
    import bs4
    import requests
    import urllib.request
    from selenium import webdriver
    from urllib import parse
    import time
    import re
    from selenium.webdriver.common.keys import Keys
    import pandas as pd

    # 중복 제거
    def duplicate(my_list):
        my_set = set(my_list) # 집합으로 변환
        new_list = list(my_set) # 리스트로 변환
        return new_list

    url = "https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=89"
    #판타지 : https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=86 V
    #현판 : https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=120 V
    #로맨스 : https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=89
    #로판 : https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=117 V
    #무협 : https://page.kakao.com/genre-total?categoryUid=11&subCategoryUid=87
    
    driver = webdriver.Chrome("C:/chromedriver.exe")
    driver.get(url)
    bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")

    SCROLL_PAUSE_TIME = 15
    title_list = []
    n=6
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    base_xpath='//*[@id="root"]/div[3]/div/div/div[{}]/div/div[2]/div[1]/div'
    while True:
        while True:
            xpath=base_xpath.format(n)
            try:
                box_area=driver.find_element_by_xpath(xpath)
            except:
                n=n+1
                break
            title_list.append(box_area.text)
            n=n+1
            
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        #time.sleep(SCROLL_PAUSE_TIME)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        time.sleep(SCROLL_PAUSE_TIME)
        #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #time.sleep(5)
        #driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(SCROLL_PAUSE_TIME)
        if new_height == last_height:
            break
        last_height = new_height


    list_num = len(title_list) # 검색 결과 개수

    for i in range(0, list_num):
        print("제목 : " + title_list[i] )

    # csv 파일로 저장
    data = pd.DataFrame(title_list, columns = ['제목'])
    data.to_csv('kakaopage_title_list5.csv', encoding='cp949', index=False)

In [12]:
kakaopage_title_extraction()

제목 : [BL] 불꽃놀이
제목 : 결혼연습
제목 : 주 비서의 이중생활
제목 : 돈보다 남자[단행본]
제목 : 피어라, 봉호리! [단행본]
제목 : 향기
제목 : 오만한 독재자
제목 : 원하고 원망해
제목 : 조선과부 상열지사
제목 : 웨딩 임파서블 [단행본]
제목 : 불건전한 채무관계
제목 : 너였으면 좋겠어
제목 : 부부의 연애
제목 : 거칠게 파고들어
제목 : 늑대 앞의 양
제목 : 불순한 청혼
제목 : 낯선 여인[할리퀸] [개정판]
제목 : 차가운 열정[할리퀸] [개정판]
제목 : 뜨거운 유혹[할리퀸] [개정판]
제목 : 루비 발레리나[할리퀸]
제목 : 선택받은 신부[할리퀸] [개정판]
제목 : 롤라의 선택[할리퀸]
제목 : 달에 홀린 늑대
제목 : 길들여 주세요
제목 : 웨딩 크래셔 [BL] [개정판]
제목 : 슈퍼스타 씨, 안티 팬입니다만? 외전 [단행본]
제목 : 이러면 안 되는 사이
제목 : 가면극 [BL]
제목 : 꽃점, 봐드립니다 [단행본]
제목 : 널 맛볼게 [단행본]
제목 : 기다리는 황제님 [단행본]
제목 : 꽃가치 [BL] [개정판]
제목 : 샤워는 조금 이따가
제목 : 사랑, 향기로 채우다
제목 : 검사와 변호사 [단행본]
제목 : 키친 [단행본][개정판]
제목 : 꿈의 정원 [단행본]
제목 : 악, 연을 품다
제목 : [BL] 인턴 [단행본]
제목 : 특별한 시작
제목 : 수면 [BL]
제목 : 다 보이는 그녀
제목 : 문제적 상속녀
제목 : 이제야 만났습니다 [개정판]
제목 : 에어프릴 로즈
제목 : 전부 너였다
제목 : 오직 너만이
제목 : 심장은 붉게 물들다 [단행본]
제목 : 박스 로맨스 플레이
제목 : 연, 사랑이라는 이유로
제목 : 이안류
제목 : 엿보지 마세요 [단행본]
제목 : 나의 무례한 첫사랑 [단행본]
제목 : 까칠하게 달콤하게 [단행본]
제목 : 헬로우 약혼녀 [단행본]
제목 : 마성의 레시피 [단행본]
제목 : 꼬리 치는 여자 [단행본]
제목 : 인생 2회 차를 샀다 [단행본]
제목 : 경성연사 [단행본]
